In [1]:
import joblib
import numpy as np
import pandas as pd

from pymatgen import Structure

from crystallus import CrystalGenerator, WyckoffCfgGenerator, WyckoffDB, SpaceGroupDB

# user-friendly print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
cfgs = joblib.load('cfgs.pkl.z')

In [3]:
composition = 'Ti4O8'
estimated_volume = 146.706
estimated_variance = 20
sp_num = 12

In [4]:
cg = CrystalGenerator(sp_num, estimated_volume, estimated_variance)
cg

CrystalGenerator(            
    spacegroup_num=12,            
    estimated_volume=146.706,            
    estimated_variance=20,            
     min_distance_tolerance=0.15,            
    angle_range=(30.0, 150.0),            
    angle_tolerance=20.0,            
    max_recurrent=5000,            
    n_jobs=-1            
)

In [5]:
len(cfgs[sp_num])

2250

In [ ]:
%%time

ret = cg.gen_many(10, *cfgs[sp_num])

In [46]:
%%time

len(ret)
joblib.dump(ret, f"{composition}_space_group_{sp_num}.pkl.z")

CPU times: user 3min 40s, sys: 641 ms, total: 3min 41s
Wall time: 3min 41s


['Ti4O8_space_group_12.pkl.z']

In [23]:
for i, tmp in enumerate(ret):
    s = Structure(lattice=tmp['lattice'], species=tmp['species'], coords=np.asarray(tmp['coords']).reshape(-1, 3))
    s.to(fmt='cif', filename=f'generated_cifs/{i}.cif', symprec=0.01)

---------

In [41]:
structure_table = pd.read_pickle('Ti4O8_structure_table_old.pd.xz')
structure_table

,structure,volume,wy_letters,abs_volume_error,expect_spacegroup_num,spacegroup_num,composition,formula,reduced_formula
0,[[1.88837334e+00 0.00000000e+00 1.15629519e-16...,101.836993,"['b', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'c', ...",43.922498,62,62,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2
1,"[[0. 0. 4.27747382] Ti, [1.875...",178.289220,"['b', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'c', ...",32.529729,62,62,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2
2,[[1.78799577e+00 0.00000000e+00 1.09483165e-16...,159.290015,"['b', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'c', ...",13.530524,62,62,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2
3,[[5.50715331e-16 3.42458538e+00 2.09695376e-16...,217.680693,"['b', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'c', ...",71.921202,62,62,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2
4,[[1.72132126e+00 0.00000000e+00 1.05400529e-16...,102.555776,"['b', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'c', ...",43.203715,62,62,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2
...,...,...,...,...,...,...,...,...,...
343807,"[[3.86475998 1.27962131 0. ] Ti, [ 3.86...",114.654707,"['e', 'e', 'e', 'e', 'h', 'h', 'h', 'h', 'f', ...",31.104784,12,12,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2
343808,"[[ 9.93926397 0.47687515 -2.25417177] Ti, [ 9...",167.169318,"['f', 'f', 'f', 'f', 'h', 'h', 'h', 'h', 'e', ...",21.409827,12,12,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2
343809,"[[ 2.20488723 -2.31374092 0. ] Ti, [ 6...",216.465334,"['e', 'e', 'e', 'e', 'h', 'h', 'h', 'h', 'f', ...",70.705843,12,12,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2
343810,"[[ 7.11999218 -3.10852331 0. ] Ti, [ 2...",187.014163,"['e', 'e', 'e', 'e', 'h', 'h', 'h', 'h', 'f', ...",41.254672,12,12,"{'Ti': 4.0, 'O': 8.0}",Ti4 O8,TiO2


In [42]:
tmp = structure_table.groupby('spacegroup_num')['wy_letters'].value_counts()

In [ ]:
from matplotlib import pyplot as plt

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8),dpi=100)
structure_table.shape
ax = structure_table.groupby('spacegroup_num').count().sort_values('formula', ascending=False).plot.bar(y=['structure'], ax=ax1)
ax.text(25,190000,'Ca2C2O6\nsize: 624899', fontsize=15, ha='right')

ax = structure_table.volume.hist(ax=ax2)
ax.xaxis.grid(False)
ax.grid(linestyle='--', linewidth=1, axis='y')
ax.set_xlabel('volume')

plt.tight_layout()